<a href="https://colab.research.google.com/github/chloebh9/Recommendation-system/blob/main/L07_1_Deep_Learning_for_Recommendation_Practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')
print(device)

cpu


In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

--2024-06-01 04:20:14--  https://files.grouplens.org/datasets/movielens/ml-latest-small.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 978202 (955K) [application/zip]
Saving to: ‘ml-latest-small.zip’

ml-latest-small.zip 100%[===================>] 955.28K  4.00MB/s    in 0.2s    

2024-06-01 04:20:15 (4.00 MB/s) - ‘ml-latest-small.zip’ saved [978202/978202]

Archive:  ml-latest-small.zip
   creating: ml-latest-small/
  inflating: ml-latest-small/links.csv  
  inflating: ml-latest-small/tags.csv  
  inflating: ml-latest-small/ratings.csv  
  inflating: ml-latest-small/README.txt  
  inflating: ml-latest-small/movies.csv  


In [ ]:
import torch

users = []
items = []
ratings = []

with open("ml-latest-small/ratings.csv", "r") as f:
    print(f.readline())

    for line in f:
        uid, mid, rating, _ = line.split(",")
        users.append(int(uid))
        items.append(int(mid))
        ratings.append(float(rating))

users = torch.tensor(users)
items = torch.tensor(items)
ratings = torch.tensor(ratings)

userId,movieId,rating,timestamp



In [ ]:
print(users)
print(items)
print(ratings)

tensor([  1,   1,   1,  ..., 610, 610, 610])
tensor([     1,      3,      6,  ..., 168250, 168252, 170875])
tensor([4., 4., 4.,  ..., 5., 5., 3.])


In [ ]:
shuffled_indices = torch.randperm(len(users))   #인덱스로 사용

train_size = int(len(users) * 0.8)
train_indices = shuffled_indices[:train_size]
test_indices = shuffled_indices[train_size:]

users_train = users[train_indices].to(device)
items_train = items[train_indices].to(device)
ratings_train = ratings[train_indices].to(device)

users_test = users[test_indices].to(device)
items_test = items[test_indices].to(device)
ratings_test = ratings[test_indices].to(device)

## Neural Collaborative Filtering

In [ ]:
import torch.nn as nn

dropout_rate = 0.5
n_factors = 10
n_users = max(users) + 1
n_items = max(items) + 1

mean = ratings_train.mean()
P = torch.randn(n_users, n_factors, requires_grad=True, device=device)
Q = torch.randn(n_items, n_factors, requires_grad=True, device=device)

model = nn.Sequential(
    nn.Linear(n_factors * 2, 10),
    nn.ReLU(),
    nn.Dropout(dropout_rate),
    nn.Linear(10, 5),
    nn.ReLU(),
    nn.Dropout(dropout_rate),
    nn.Linear(5, 1)
).to(device)

In [ ]:
mse = nn.MSELoss()

optim = torch.optim.Adam(list(model.parameters()) + [P, Q], lr=0.1)

for epoch in range(3001):
    model.train()
    x = torch.cat((P[users_train], Q[items_train]), dim=1)
    h = model(x)

    cost = mse(h, ratings_train.view(-1,1))

    optim.zero_grad()
    cost.backward()
    optim.step()

    with torch.no_grad():
        if epoch % 100 == 0:
            model.eval()

            x_train = torch.cat((P[users_train], Q[items_train]), dim=1)
            h_train = model(x_train)
            train_cost = mse(h_train, ratings_train.view(-1,1))

            x_test = torch.cat((P[users_test], Q[items_test]), dim=1)
            h_test = model(x_test)
            test_cost = mse(h_test, ratings_test.view(-1,1))

            print(f"epoch: {epoch}, train_mse: {train_cost.item()}, test_mse: {test_cost.item()}")

 ## AutoRec

In [ ]:
rating_matrix_train = torch.zeros(n_users, n_items)
for uid, mid, rating in zip(users_train, items_train, ratings_train):
    rating_matrix_train[uid, mid] = rating
rating_matrix_train = rating_matrix_train.to(device)

rating_matrix_test = torch.zeros(n_users, n_items)
for uid, mid, rating in zip(users_test, items_test, ratings_test):
    rating_matrix_test[uid, mid] = rating
rating_matrix_test = rating_matrix_test.to(device)

In [ ]:
import torch.nn as nn

n_factors = 500

model = nn.Sequential(
    nn.Linear(n_users, n_factors),
    nn.Sigmoid(),
    nn.Linear(n_factors, n_users)
).to(device)

In [ ]:
optim = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=0.001)

mse = nn.MSELoss()

non_zeros_train = rating_matrix_train != 0
non_zeros_test = rating_matrix_test != 0

for epoch in range(1001):
    h = model(rating_matrix_train.T)
    cost = mse(h.T[non_zeros_train], rating_matrix_train[non_zeros_train])
    optim.zero_grad()
    cost.backward()
    optim.step()

    with torch.no_grad():
        if epoch % 10 == 0:
            h_train = model(rating_matrix_train.T)
            train_mse = mse(h_train.T[non_zeros_train], rating_matrix_train[non_zeros_train])
            h_test = model(rating_matrix_test.T)
            test_mse = mse(h_test.T[non_zeros_test], rating_matrix_test[non_zeros_test])

            print(f"epoch: {epoch}, train_mse: {train_mse.item()}, test_mse: {test_mse.item()}")